In [23]:
import geopandas
import pandas as pd

OPEN_AREAS_STRING = "openAreas"
URBAN_AREAS_STRING = "urbanAreas"
WOODS_STRING = "woods"
WATERS_STRING = "waters"
MOUNTAINS_STRING = "mountains"

In [24]:
# area 
mybbox = (714000.0,5323000.0,715000.0,5324000.0)

In [25]:
df = geopandas.read_file("enveloped_polygons.shp",bbox = mybbox, include_fields=["OBJART_TXT"])
preprocessedDf = pd.DataFrame(df)

In [26]:
from functools import partial
import pyproj
from shapely.ops import transform

# calculate centroid of a polygon and transforms coordinates from epsg25832 to epsg4326
def transform_coordinates(row):
    row = row.centroid
    project = partial(
    pyproj.transform,
    pyproj.Proj('epsg:25832'),
    pyproj.Proj('epsg:4326'))
    row = transform(project, row)
    return [row.x,row.y]

# add flight altitude based on the area
def add_altitude(row):
    areaType = row['OBJART_TXT']
    if areaType == OPEN_AREAS_STRING:
        return row['centre'].append(70)
    if areaType == URBAN_AREAS_STRING:
        return row['centre'].append(90)
    if areaType == WOODS_STRING:
        return row['centre'].append(80)
    if areaType == WATERS_STRING:
        return row['centre'].append(60)
    if areaType == MOUNTAINS_STRING:
        return row['centre'].append(100)
    
    return row



In [27]:
preprocessedDf['centre'] = preprocessedDf['geometry'].apply(lambda row: transform_coordinates(row))
preprocessedDf.apply(add_altitude,axis=1)
preprocessedDf = preprocessedDf.drop(['geometry'], axis=1)

C:\Users\Oliver\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\shapely\ops.py:274: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return type(geom)(zip(*func(*zip(*geom.coords))))
C:\Users\Oliver\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\shapely\ops.py:274: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return type(geom)(zip(*func(*zip(*geom.coords))))
C:\Users\Oliver\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\shapely\ops.py:274: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  

,OBJART_TXT,centre
0,urbanAreas,"[48.03186597519473, 11.878701435772102, 90]"
1,openAreas,"[48.02278641363004, 11.873498771538323, 70]"
2,openAreas,"[48.03062063547658, 11.880138430124077, 70]"
3,openAreas,"[48.03264234099565, 11.881402512218557, 70]"
4,openAreas,"[48.02285086769221, 11.875040489331514, 70]"


In [28]:
areaDict = dict()
urbanIndex = 0
woodsIndex = 0
watersIndex = 0
openIndex = 0
mountainsIndex = 0
for index, row in preprocessedDf.iterrows():
    currObj = row['OBJART_TXT']
    
    if currObj == URBAN_AREAS_STRING:
        areaDict[str(row['OBJART_TXT']).lower() + str(urbanIndex)] = row['centre']
        urbanIndex += 1
    elif currObj == WOODS_STRING:
        areaDict[str(row['OBJART_TXT']).lower() + str(woodsIndex)] = row['centre']
        woodsIndex += 1
    elif currObj == WATERS_STRING:
        areaDict[str(row['OBJART_TXT']).lower() + str(watersIndex)] = row['centre']
        watersIndex += 1
    elif currObj == OPEN_AREAS_STRING:
        areaDict[str(row['OBJART_TXT']).lower() + str(openIndex)] = row['centre']
        openIndex += 1
    elif currObj == MOUNTAINS_STRING:
        areaDict[str(row['OBJART_TXT']).lower() + str(mountainsIndex)] = row['centre']
        mountainsIndex += 1

In [29]:
# generate pddl problem file from 
with open("generated_uav_instance.pddl", "w") as file:
    # Writing data to a file
    file.write("(define (problem uav_generated_problem) (:domain uav) \n")
    file.write("(:objects \n")
    file.write("myUAV - uav \n")
    for currArea in areaDict.keys():
        file.write(currArea + " ")
    file.write("home -waypoint\n)\n")
    file.write("(:init\n")
    file.write("(landed myUAV)\n")
    file.write("(at myUAV home)\n")
    file.write(")\n\n")
    file.write("(:goal (and\n\n")
    file.write("(landed myUAV)\n")
    file.write("))\n)")

    #file.writelines(L)

In [30]:
# write to json
import json
with open('lookupTable.json', 'w') as fp:
    json.dump(areaDict, fp, indent=4)


In [31]:
# write to yaml
import yaml
with open('lookupTable.json') as json_file:
    data = json.load(json_file)
    print(data)

with open('lookupTable.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

{'urbanareas0': [48.03186597519473, 11.878701435772102, 90], 'openareas0': [48.02278641363004, 11.873498771538323, 70], 'openareas1': [48.03062063547658, 11.880138430124077, 70], 'openareas2': [48.03264234099565, 11.881402512218557, 70], 'openareas3': [48.02285086769221, 11.875040489331514, 70], 'openareas4': [48.02673597356748, 11.872015486372552, 70], 'openareas5': [48.031001892747895, 11.886817292676243, 70], 'openareas6': [48.02889910623069, 11.883232534176702, 70], 'openareas7': [48.02952247970669, 11.883502815433877, 70], 'openareas8': [48.028346743605795, 11.875320542034016, 70], 'openareas9': [48.03033125196925, 11.880913954388413, 70], 'openareas10': [48.029574018116776, 11.876203013531065, 70], 'openareas11': [48.02830369357392, 11.873833878044053, 70], 'openareas12': [48.03130609931262, 11.87037401463903, 70], 'openareas13': [48.03244848435939, 11.878033493130944, 70], 'openareas14': [48.030530230404146, 11.88217833407476, 70], 'openareas15': [48.033191805971335, 11.87296243